In [9]:
import random
import math
import sys

sys.setrecursionlimit(10**6)

class Checkers():
    def __init__(self):
        self.initial = Checkers_Board()

    def remove_self_intersections(self, i, board):
        # There are 4 possible moves.
        turn = board.to_move
        if(turn == 'W'):
            piece = board.P[turn][i]
            moves = []
            if(piece[1] < board.dim - 1):
                if(piece[0] > 0):
                    moves.append((piece[0] - 1, piece[1] + 1))
                if(piece[0] < board.dim - 1):
                    moves.append((piece[0] + 1, piece[1] + 1))
            if(board.Kings[turn][i] and piece[1] > 0):
                if(piece[0] > 0):
                    moves.append((piece[0] - 1, piece[1] - 1))
                if(piece[0] < board.dim - 1):
                    moves.append((piece[0] + 1, piece[1] - 1))
            available_moves = []
            for m in moves:
                if m in board.P['W']:
                    continue
                available_moves.append(m)
            return available_moves
            
        if(turn == 'B'):
            piece = board.P[turn][i]
            moves = []
            if(piece[1] > 0 ):
                if(piece[0] > 0):
                    moves.append((piece[0] - 1, piece[1] - 1))
                if(piece[0] < board.dim - 1):
                    moves.append((piece[0] + 1, piece[1] - 1))
            if(board.Kings[turn][i] and piece[1] < board.dim - 1):
                if(piece[0] > 0):
                    moves.append((piece[0] - 1, piece[1] + 1))
                if(piece[0] < board.dim - 1):
                    moves.append((piece[0] + 1, piece[1] + 1))
            available_moves = []
            for m in moves:
                if m in board.P['B']:
                    continue
                available_moves.append(m)
            return available_moves

    def capture_cycle(self, position, captured, board):
        directions = []
        p = (position[0] - 1, position[1] - 1)
        if p[0] in range(board.dim) and p[1] in range(board.dim): directions.append(p)
        p = (position[0] + 1, position[1] - 1)
        if p[0] in range(board.dim) and p[1] in range(board.dim): directions.append(p)
        p = (position[0] - 1, position[1] + 1)
        if p[0] in range(board.dim) and p[1] in range(board.dim): directions.append(p)
        p = (position[0] + 1, position[1] + 1)
        if p[0] in range(board.dim) and p[1] in range(board.dim): directions.append(p)
        available_directions = []
        for d in directions:
            # we don't want to recapture and we can not do anything
            # if one of our pieces is  in the way. 
            if d not in captured and d not in board.P[board.to_move]:
                available_directions.append(d)
        capture_directions = 0
        for d in available_directions:
            rival = 'B' if board.to_move == 'W' else 'W'
            if d in board.P[rival]:
                if d in captured: continue
                # we have a rival piece, now we need to see if we can jump it.
                direction = (d[0] - position[0], d[1] - position[1])
                destination = (d[0] + direction[0], d[1] + direction[1])
                if destination[0] not in range(board.dim) or destination[1] not in range(board.dim): continue
                # We also need to check whether the destination is open. That is, clear.
                if destination in board.P['W'] or destination in board.P['B']: continue
                # At this point we have a valid square to land on.
                # We need to add the piece to captured and call capture_cycle again
                captured.append(d)
                capture_directions += 1
                self.capture_cycle(destination, captured, board)
        if(len(captured) > 0):
            captured.append(None)
            captured.append(position)
            captured.append(None)
                 
    
    def generate_capture_actions(self, i, piece, moves, board):
        captured = []
        other_player = 'B' if board.to_move == 'W' else 'W'
        other_player_piece_locations = board.P[other_player]
        best_capture = None
        for move in moves:
            # can we take a piece?
            if move in other_player_piece_locations:
                # We need to check whether the destination is valid
                direction = (move[0] - piece[0], move[1] - piece[1])
                destination = (move[0] + direction[0], move[1] + direction[1])
                if destination[0] not in range(board.dim) or destination[1] not in range(board.dim): continue
                # We also need to check whether the destination is open. That is, clear.
                if destination in board.P['W'] or destination in board.P['B']: continue
                # At this point we have a valid square to land on.
                captures = [move]
                final_destination = None
                self.capture_cycle(destination, captures, board)
                captures = list(dict.fromkeys(captures))
                if best_capture == None:
                    best_capture = captures
                else:
                    if(len(captures) > len(best_capture)):
                        best_capture = captures
        return [i, best_capture]

    # action is a tuple:
    # (i, [taken_pieces, None, final_position], make_king)
    def actions(self, board):
        actions = []
        for i in range(board.piece_count):
            turn = board.to_move
            piece = board.P[turn][i]
            # If the piece is taken, just continue.
            if piece == None: continue
            moves = self.remove_self_intersections(i, board)
            # If there are no moves ,just continue.
            if(len(moves) == 0): continue
            action = self.generate_capture_actions(i, piece, moves, board)
            actions.append(action)
        capture_only_actions = []
        for a in actions:
            if a[1] != None:
                capture_only_actions.append(a)
        if len(capture_only_actions) > 0:
            return capture_only_actions

        # if capture_only_actions is of size zero, 
        # we need to look into just moving pieces
        actions = []
        for i in range(board.piece_count):
            turn = board.to_move
            piece = board.P[turn][i]
            # If the piece is taken, just continue.
            if piece == None: continue
            moves = self.remove_self_intersections(i, board)
            legal_moves = []
            other_player = 'B' if board.to_move == 'W' else 'W'
            for move in moves:
                if move in board.P[other_player]:
                    continue
                legal_moves.append(move)
            for move in legal_moves:
                actions.append([i, [None, move]])
        #random.shuffle(actions)
        return actions

    def promote_king(self, piece_index, board):
        piece = board.P[board.to_move][piece_index]
        if board.to_move == 'W':
            if piece[1] == board.dim - 1:
                board.Kings['W'][piece_index] = True
        if board.to_move == 'B':
            if piece[1] == 0:
                board.Kings['B'][piece_index] = True       
    
    def result(self, board, action):
        # create a copy of the current board
        new_board = board.new()
        piece_index = action[0]
        move_data = action[1]
        k = move_data.index(None)
        new_board.P[new_board.to_move][piece_index] = move_data[k+1]
        # check if promotion is needed
        self.promote_king(piece_index, new_board)
        # if there is no captures, exit
        if k == 0:
            new_board.to_move = 'B' if new_board.to_move == 'W' else 'W'
            return new_board
        # else we have to register captures
        captured = move_data[:k]
        other_player = 'B' if new_board.to_move == 'W' else 'W'
        for c in captured:
            c_index = new_board.P[other_player].index(c)
            new_board.P[other_player][c_index] = None
        new_board.to_move = 'B' if new_board.to_move == 'W' else 'W'
        return new_board

    def is_terminal(self, board):
        no_whites = board.P['W'].count(None) == board.piece_count
        no_blacks = board.P['B'].count(None) == board.piece_count
        return no_whites or no_blacks

    def utility(self, board):
        player = board.to_move
        print("TERMINAL STATE REACHED")
        return 1 if player == 'W' else -1


class Checkers_Board():
    def __init__(self):
        # initial positions
        # 8x8
        #self.dim = 8
        #W = [(x,y) for y in range(0,3) for x in range(y%2,8,2)]
        #B = [(x,y) for y in range(5,8) for x in range(y%2,8,2)]
        # 6x6
        #self.dim = 6
        #W = [(x,y) for y in range(0,2) for x in range(y%2,6,2)]
        #B = [(x,y) for y in range(4,6) for x in range(y%2,6,2)]
        # 4x4
        self.dim = 4
        #W = [(0,0), (2,0)]
        #B = [(1,3), (3,3)]
        W = [(0,0)]
        B = [(1,3)]
        self.piece_count = len(W)
        self.P = {'W' : W, 'B' : B}
        # we record king status as a mapping from location to boolean
        W_King = [False for i in range(self.piece_count)]
        B_King = [False for i in range(self.piece_count)]
        self.Kings = {'W' : W_King,'B' : B_King}
        self.to_move = 'W'

    def new(self):
        new_board = Checkers_Board()
        for i in range(self.piece_count):
            new_board.P['W'][i] = self.P['W'][i]
            new_board.P['B'][i] = self.P['B'][i]
            new_board.Kings['W'][i] = self.Kings['W'][i]
            new_board.Kings['B'][i] = self.Kings['B'][i]
        new_board.piece_count = self.piece_count
        new_board.to_move = self.to_move
        return new_board

    def utility(self):
        player = self.to_move
        print("TERMINAL STATE REACHED")
        return 1 if player == 'W' else -1

infinity = math.inf

def alphabeta_search(game, state):
    def max_value(state, alpha, beta):
        player = state.to_move       
        if game.is_terminal(state):
            return game.utility(state), None
        v, move = -infinity, None
        print("AVAILABLE, ACTIONS: ", game.actions(state))
        for a in game.actions(state):
            print("MAX ", player, a)
            v2, _ = min_value(game.result(state, a), alpha, beta)
            if v2 > v:
                v, move = v2, a
                alpha = max(alpha, v)
            if v >= beta:
                return v, move
        return v, move

    def min_value(state, alpha, beta):
        player = state.to_move
        if game.is_terminal(state):
            return game.utility(state), None
        v, move = +infinity, None
        print("AVAILABLE, ACTIONS: ", game.actions(state))
        for a in game.actions(state):
            print("MIN ", player, a)
            v2, _ = max_value(game.result(state, a), alpha, beta)
            if v2 < v:
                v, move = v2, a
                beta = min(beta, v)
            if v <= alpha:
                return v, move
        return v, move

    return max_value(state, -infinity, +infinity)

def play_game(game, strategies: dict, verbose=False):
    """Play a turn-taking game. `strategies` is a {player_name: function} dict,
    where function(state, game) is used to get the player's move."""
    state = game.initial
    while not game.is_terminal(state):
        player = state.to_move
        move = strategies[player](game, state)
        state = game.result(state, move)
        if verbose: 
            print('Player', player, 'move:', move)
            print(state)
    return state

def random_player(game, state): 
    selection =  random.choice(list(game.actions(state)))
    print("\t\t: ", selection)
    return selection

def player(search_algorithm):
    """A game player who uses the specified search algorithm"""
    return lambda game, state: search_algorithm(game, state)[1]

play_game(Checkers(), dict(W=random_player, B=player(alphabeta_search)), verbose=True).utility()
#play_game(Checkers(), dict(B=random_player, W=random_player), verbose=True).utility()

		:  [0, [None, (1, 1)]]
Player W move: [0, [None, (1, 1)]]
AVAILABLE, ACTIONS:  [[0, [None, (0, 2)]], [0, [None, (2, 2)]]]
MAX  B [0, [None, (0, 2)]]
AVAILABLE, ACTIONS:  [[0, [None, (2, 2)]]]
MIN  W [0, [None, (2, 2)]]
AVAILABLE, ACTIONS:  [[0, [None, (1, 1)]]]
MAX  B [0, [None, (1, 1)]]
AVAILABLE, ACTIONS:  [[0, [None, (1, 3)]], [0, [None, (3, 3)]]]
MIN  W [0, [None, (1, 3)]]
AVAILABLE, ACTIONS:  [[0, [None, (0, 0)]], [0, [None, (2, 0)]]]
MAX  B [0, [None, (0, 0)]]
AVAILABLE, ACTIONS:  [[0, [None, (0, 2)]], [0, [None, (2, 2)]]]
MIN  W [0, [None, (0, 2)]]
AVAILABLE, ACTIONS:  [[0, [None, (1, 1)]]]
MAX  B [0, [None, (1, 1)]]
AVAILABLE, ACTIONS:  [[0, [(1, 1), None, (2, 0)]]]
MIN  W [0, [(1, 1), None, (2, 0)]]
TERMINAL STATE REACHED
MIN  W [0, [None, (2, 2)]]
AVAILABLE, ACTIONS:  [[0, [None, (1, 1)]]]
MAX  B [0, [None, (1, 1)]]
AVAILABLE, ACTIONS:  [[0, [(1, 1), None, (0, 0)]]]
MIN  W [0, [(1, 1), None, (0, 0)]]
TERMINAL STATE REACHED
MAX  B [0, [None, (2, 0)]]
AVAILABLE, ACTIONS:  [[0

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fcb806dbd90>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
